## Emulator API 开发笔记

In [ ]:
from seedemu import *
import os, sys

HOSTS_PER_AS = 2

###############################################################################
# Set the platform information
# script_name = os.path.basename(__file__)

# if len(sys.argv) == 1:
#     platform = Platform.AMD64
# elif len(sys.argv) == 2:
#     if sys.argv[1].lower() == 'amd':
#         platform = Platform.AMD64
#     elif sys.argv[1].lower() == 'arm':
#         platform = Platform.ARM64
#     else:
#         print(f"Usage:  {script_name} amd|arm")
#         sys.exit(1)
# else:
#     print(f"Usage:  {script_name} amd|arm")
#     sys.exit(1)

emu   = Emulator()
ebgp  = Ebgp()
base  = Base()
ovpn  = OpenVpnRemoteAccessProvider()

###############################################################################
# Create internet exchanges
ix100 = base.createInternetExchange(100)
ix101 = base.createInternetExchange(101)
ix102 = base.createInternetExchange(102)
ix103 = base.createInternetExchange(103)
ix104 = base.createInternetExchange(104)
ix105 = base.createInternetExchange(105)

# Customize names (for visualization purpose)
ix100.getPeeringLan().setDisplayName('NYC-100')
ix101.getPeeringLan().setDisplayName('San Jose-101')
ix102.getPeeringLan().setDisplayName('Chicago-102')
ix103.getPeeringLan().setDisplayName('Miami-103')
ix104.getPeeringLan().setDisplayName('Boston-104')
ix105.getPeeringLan().setDisplayName('Huston-105')


###############################################################################
# Create Transit Autonomous Systems 

## Tier 1 ASes
Makers.makeTransitAs(base, 2, [100, 101, 102, 105], 
        [(100, 101), (101, 102), (100, 105)] 
)

Makers.makeTransitAs(base, 3, [100, 103, 104, 105], 
        [(100, 103), (100, 105), (103, 105), (103, 104)]
)

Makers.makeTransitAs(base, 4, [100, 102, 104], 
        [(100, 104), (102, 104)]
)

## Tier 2 ASes
Makers.makeTransitAs(base, 11, [102, 105], [(102, 105)])
Makers.makeTransitAs(base, 12, [101, 104], [(101, 104)])


###############################################################################
# Create single-homed stub ASes. 
Makers.makeStubAsWithHosts(emu, base, 150, 100, HOSTS_PER_AS)
Makers.makeStubAsWithHosts(emu, base, 151, 100, HOSTS_PER_AS)
Makers.makeStubAsWithHosts(emu, base, 152, 101, HOSTS_PER_AS)
Makers.makeStubAsWithHosts(emu, base, 153, 101, HOSTS_PER_AS)
Makers.makeStubAsWithHosts(emu, base, 154, 102, HOSTS_PER_AS)
Makers.makeStubAsWithHosts(emu, base, 160, 103, HOSTS_PER_AS)
Makers.makeStubAsWithHosts(emu, base, 161, 103, HOSTS_PER_AS)
Makers.makeStubAsWithHosts(emu, base, 162, 103, HOSTS_PER_AS)
Makers.makeStubAsWithHosts(emu, base, 163, 104, HOSTS_PER_AS)
Makers.makeStubAsWithHosts(emu, base, 164, 104, HOSTS_PER_AS)
Makers.makeStubAsWithHosts(emu, base, 170, 105, HOSTS_PER_AS)
Makers.makeStubAsWithHosts(emu, base, 171, 105, HOSTS_PER_AS)

# An example to show how to add a host with customized IP address
as154 = base.getAutonomousSystem(154)
as154.getNetwork("net0").enableRemoteAccess(ovpn)
as154.createNetwork("net1")
router1 = as154.createRouter('router1')
router1.joinNetwork('net0')
router1.joinNetwork("net1")
for counter in range(HOSTS_PER_AS):
    name = 'host_{}'.format(counter + 2)
    host = as154.createHost(name)
    host.joinNetwork("net1")


as163 = base.getAutonomousSystem(163)
as163.getNetwork('net0').enableRemoteAccess(ovpn)
new_host = as154.createHost('host_new').joinNetwork('net0', address = '10.154.0.129')
as2 = base.getAutonomousSystem(2)
new_host_2 = as2.createHost('host_new_2').joinNetwork('net_101_102', address = '10.2.1.129')
from seedemu.core import OptionRegistry, OptionMode


o = OptionRegistry().sysctl_netipv4_conf_rp_filter({'all': False, 'default': False, 'net0': False}, mode = OptionMode.RUN_TIME)
new_host.setOption(o)

o = OptionRegistry().sysctl_netipv4_udp_rmem_min(5000, mode = OptionMode.RUN_TIME)
new_host.setOption(o)

###############################################################################
# Peering via RS (route server). The default peering mode for RS is PeerRelationship.Peer, 
# which means each AS will only export its customers and their own prefixes. 
# We will use this peering relationship to peer all the ASes in an IX.
# None of them will provide transit service for others. 

ebgp.addRsPeers(100, [2, 3, 4])
ebgp.addRsPeers(102, [2, 4])
ebgp.addRsPeers(104, [3, 4])
ebgp.addRsPeers(105, [2, 3])

# To buy transit services from another autonomous system, 
# we will use private peering  

ebgp.addPrivatePeerings(100, [2],  [150, 151], PeerRelationship.Provider)
ebgp.addPrivatePeerings(100, [3],  [150], PeerRelationship.Provider)

ebgp.addPrivatePeerings(101, [2],  [12], PeerRelationship.Provider)
ebgp.addPrivatePeerings(101, [12], [152, 153], PeerRelationship.Provider)

ebgp.addPrivatePeerings(102, [2, 4],  [11, 154], PeerRelationship.Provider)
ebgp.addPrivatePeerings(102, [11], [154], PeerRelationship.Provider)

ebgp.addPrivatePeerings(103, [3],  [160, 161, 162], PeerRelationship.Provider)

ebgp.addPrivatePeerings(104, [3, 4], [12], PeerRelationship.Provider)
ebgp.addPrivatePeerings(104, [4],  [163], PeerRelationship.Provider)
ebgp.addPrivatePeerings(104, [12], [164], PeerRelationship.Provider)

ebgp.addPrivatePeerings(105, [3],  [11, 170], PeerRelationship.Provider)
ebgp.addPrivatePeerings(105, [11], [171], PeerRelationship.Provider)


###############################################################################
# Add layers to the emulator

emu.addLayer(base)
emu.addLayer(Routing())
emu.addLayer(ebgp) 
emu.addLayer(Ibgp())
emu.addLayer(Ospf())

# Create a DHCP server (virtual node).
dhcp = DHCPService()

# Default DhcpIpRange : x.x.x.101 ~ x.x.x.120
# Set DhcpIpRange : x.x.x.125 ~ x.x.x.140
dhcp.install('dhcp-01').setIpRange(125, 140)
dhcp.install('dhcp-02')


# Customize the display name (for visualization purpose)
emu.getVirtualNode('dhcp-01').setDisplayName('DHCP Server 1')
emu.getVirtualNode('dhcp-02').setDisplayName('DHCP Server 2')


# Create new hosts in AS-151 and AS-161, use them to host the DHCP servers.
# We can also host it on an existing node.
as151 = base.getAutonomousSystem(151)
as151.createHost('dhcp-server-01').joinNetwork('net0')

as161 = base.getAutonomousSystem(161)
as161.createHost('dhcp-server-02').joinNetwork('net0')

# Bind the DHCP virtual node to the physical node.
emu.addBinding(Binding('dhcp-01', filter = Filter(asn=151, nodeName='dhcp-server-01')))
emu.addBinding(Binding('dhcp-02', filter = Filter(asn=161, nodeName='dhcp-server-02')))


# Create new hosts in AS-151 and AS-161
# Make them to use dhcp instead of static ip
as151.createHost('dhcp-client-01').joinNetwork('net0', address = "dhcp")
as151.createHost('dhcp-client-02').joinNetwork('net0', address = "dhcp")

as161.createHost('dhcp-client-03').joinNetwork('net0', address = "dhcp")
as161.createHost('dhcp-client-04').joinNetwork('net0', address = "dhcp")

# Add the dhcp layer
emu.addLayer(dhcp)

# Render the emulation
emu.render()
# # Attach the Internet Map container to the emulator
# docker = Docker(platform=platform)
# emu.compile(docker, './output', override=True)

== Emulator: requesting configure: Base
== Emulator: entering Base...
== Emulator: invoking pre-configure hooks for Base...
== Emulator: configuring Base...
==== BaseLayer: registering nodes...
==== OpenVpnRemoteAccessProvider: setting up OpenVPN remote access for net0 in AS154...
==== OpenVpnRemoteAccessProvider: setting up OpenVPN remote access for net0 in AS163...
==== BaseLayer: setting up internet exchanges...
==== BaseLayer: setting up autonomous systems...
== Emulator: invoking post-configure hooks for Base...
== Emulator: done: Base
== Emulator: collecting virtual node names in the emulation...
== Emulator: found 2 virtual nodes.
== Emulator: resolving binding for all virtual nodes...
==== Binding: dhcp-01: looking for binding for dhcp-01
==== Binding: dhcp-01: trying node as2/host_new_2...
==== Binding: dhcp-01: node asn (2) != filter asn (151), trying next node.
==== Binding: dhcp-01: trying node as2/r100...
==== Binding: dhcp-01: node asn (2) != filter asn (151), trying next

### 1. Emulator 的 Registry 信息

In [ ]:
print(emu.getRegistry())

Registry:
    Object seedemu/dict/layersdb:
        LayerDatabase
    Object seedemu/list/bindingdb:
        BindingDatabase
    Object seedemu/layer/Base:
        BaseLayer:
            AutonomousSystems:
                AutonomousSystem 2:
                    Networks:
                        Network net_100_101 (NetworkType.Local):
                            Prefix: 10.2.0.0/24
                            AddressAssignmentConstraint: Default Constraint
                        Network net_101_102 (NetworkType.Local):
                            Prefix: 10.2.1.0/24
                            AddressAssignmentConstraint: Default Constraint
                        Network net_100_105 (NetworkType.Local):
                            Prefix: 10.2.2.0/24
                            AddressAssignmentConstraint: Default Constraint
                    Routers:
                        Node r100:
                            Role: NodeRole.Router
                            Interfaces:
       

### 2. 获取网络所有 AS

Base 类有 API 接口 `getAsns()` 可以获取网络所有 AS Number，通过接口 `getAutonomousSystem(asn)` 可以获取具体的 AS 对象。

In [ ]:
print(base.getAsns())

print(base.getAutonomousSystem(12))

[2, 3, 4, 11, 12, 150, 151, 152, 153, 154, 160, 161, 162, 163, 164, 170, 171]
AutonomousSystem 12:
    Networks:
        Network net_101_104 (NetworkType.Local):
            Prefix: 10.12.0.0/24
            AddressAssignmentConstraint: Default Constraint
    Routers:
        Node r101:
            Role: NodeRole.Router
            Interfaces:
                Interface:
                    Connected to: ix101
                    Address: 10.101.0.12
                    Link Properties: 10.101.0.12
                        Added Latency: 0 ms
                        Egress Bandwidth Limit: unlimited bps
                Interface:
                    Connected to: net_101_104
                    Address: 10.12.0.254
                    Link Properties: 10.12.0.254
                        Added Latency: 0 ms
                        Egress Bandwidth Limit: unlimited bps
            Files:
                /etc/bird/bird.conf:
                    > router id 10.0.0.14;
                    > ip

### 3. 获取 AS 当中所有网络

对于 AS 对象有接口 `getNetworks()` 可以获取 AS 下所有网络名字，用 `getNetwork(name)` 可以获取具体的网络对象。

In [ ]:
for asn in base.getAsns():
    __as = base.getAutonomousSystem(asn)
    print(__as.getNetworks())
    for net_name in __as.getNetworks():
        __net = __as.getNetwork(net_name)
        print(__net)

['net_100_101', 'net_101_102', 'net_100_105']
Network net_100_101 (NetworkType.Local):
    Prefix: 10.2.0.0/24
    AddressAssignmentConstraint: Default Constraint

Network net_101_102 (NetworkType.Local):
    Prefix: 10.2.1.0/24
    AddressAssignmentConstraint: Default Constraint

Network net_100_105 (NetworkType.Local):
    Prefix: 10.2.2.0/24
    AddressAssignmentConstraint: Default Constraint

['net_100_103', 'net_100_105', 'net_103_105', 'net_103_104']
Network net_100_103 (NetworkType.Local):
    Prefix: 10.3.0.0/24
    AddressAssignmentConstraint: Default Constraint

Network net_100_105 (NetworkType.Local):
    Prefix: 10.3.1.0/24
    AddressAssignmentConstraint: Default Constraint

Network net_103_105 (NetworkType.Local):
    Prefix: 10.3.2.0/24
    AddressAssignmentConstraint: Default Constraint

Network net_103_104 (NetworkType.Local):
    Prefix: 10.3.3.0/24
    AddressAssignmentConstraint: Default Constraint

['net_100_104', 'net_102_104']
Network net_100_104 (NetworkType.Loc

### 4. 网络相关信息查看

1. 查看网络前缀：通过 `getPrefix()` 函数查看

In [ ]:
for asn in base.getAsns():
    __as = base.getAutonomousSystem(asn)
    print("ASN: {}".format(asn))
    for net_name in __as.getNetworks():
        __net = __as.getNetwork(net_name)
        print(__net.getPrefix())

ASN: 2
10.2.0.0/24
10.2.1.0/24
10.2.2.0/24
ASN: 3
10.3.0.0/24
10.3.1.0/24
10.3.2.0/24
10.3.3.0/24
ASN: 4
10.4.0.0/24
10.4.1.0/24
ASN: 11
10.11.0.0/24
ASN: 12
10.12.0.0/24
ASN: 150
10.150.0.0/24
ASN: 151
10.151.0.0/24
ASN: 152
10.152.0.0/24
ASN: 153
10.153.0.0/24
ASN: 154
10.154.0.0/24
10.154.1.0/24
ASN: 160
10.160.0.0/24
ASN: 161
10.161.0.0/24
ASN: 162
10.162.0.0/24
ASN: 163
10.163.0.0/24
ASN: 164
10.164.0.0/24
ASN: 170
10.170.0.0/24
ASN: 171
10.171.0.0/24


2. 查看有没有 VPN 服务：可以通过 `getRemoteAccessProvider()` 接口查看，如果返回 None 就是没有，否则就是有，有就可以通过建立的 Router 来找到端口号

In [ ]:
from seedemu.core.enums import NodeRole
for asn in base.getAsns():
    __as = base.getAutonomousSystem(asn)
    for net_name in __as.getNetworks():
        __net = __as.getNetwork(net_name)
        if __net.getRemoteAccessProvider() is not None:
            print(f"Net {__net.getPrefix()} has VPN Server")
            for __node in __net.getAssociations():
                if __node.getRole() == NodeRole.Router:
                    print(__node.getPorts())
            

Net 10.154.0.0/24 has VPN Server
[]
[(65000, 1194, 'udp')]
Net 10.163.0.0/24 has VPN Server
[(65001, 1194, 'udp')]


3. 路由信息

在 AS 对象里面通过 `getRouters()` 可以获得所有路由名字，再通过 `getRouter(name)` 获得具体的路由对象，通过 `getBorderRouters()` 可以获所有边界路由对象，通过 `getInterfaces()` 可以获得所有路由的接口对象。

找到接口对象之后，利用 `getNet()` 接口就可以获得接的网络对象

In [ ]:
from seedemu.core.enums import NodeRole
for asn in base.getAsns():
    __as = base.getAutonomousSystem(asn)
    print(f"ASN: {asn}")
    for __router in __as.getBorderRouters():
        print(__router.getName())
        for __interface in __router.getInterfaces():
            print(__interface.getNet().getName())
            print(__interface.getNet().getPrefix())
            print(__interface.getAddress())
            print("========")


ASN: 2
r100
ix100
10.100.0.0/24
10.100.0.2
net_100_101
10.2.0.0/24
10.2.0.254
net_100_105
10.2.2.0/24
10.2.2.254
r101
ix101
10.101.0.0/24
10.101.0.2
net_100_101
10.2.0.0/24
10.2.0.253
net_101_102
10.2.1.0/24
10.2.1.254
r102
ix102
10.102.0.0/24
10.102.0.2
net_101_102
10.2.1.0/24
10.2.1.253
r105
ix105
10.105.0.0/24
10.105.0.2
net_100_105
10.2.2.0/24
10.2.2.253
ASN: 3
r100
ix100
10.100.0.0/24
10.100.0.3
net_100_103
10.3.0.0/24
10.3.0.254
net_100_105
10.3.1.0/24
10.3.1.254
r103
ix103
10.103.0.0/24
10.103.0.3
net_100_103
10.3.0.0/24
10.3.0.253
net_103_105
10.3.2.0/24
10.3.2.254
net_103_104
10.3.3.0/24
10.3.3.254
r104
ix104
10.104.0.0/24
10.104.0.3
net_103_104
10.3.3.0/24
10.3.3.253
r105
ix105
10.105.0.0/24
10.105.0.3
net_100_105
10.3.1.0/24
10.3.1.253
net_103_105
10.3.2.0/24
10.3.2.253
ASN: 4
r100
ix100
10.100.0.0/24
10.100.0.4
net_100_104
10.4.0.0/24
10.4.0.254
r102
ix102
10.102.0.0/24
10.102.0.4
net_102_104
10.4.1.0/24
10.4.1.254
r104
ix104
10.104.0.0/24
10.104.0.4
net_100_104
10.4.0.0/24

但是，在我的理解当中，emulator 是先建立好一层一层的对象，再通过接口去接的，只有 Router 通过 Interface 去接 Network，而没有在 Network 记录下这个 Interface，因此我们没法通过网络去获取默认路由，因此尝试添加功能

在 `core/Network.py` 添加接口 `getDefaultRouters`:

```python
def getDefaultRouters(self) -> List[IPv4Address]:
        """!
        @brief Get default routers for this network.

        @returns list of default routers.
        """
        routers = []
        for __node in self.getAssociations():
            if __node.getRole() == NodeRole.BorderRouter:
                for __interface in __node.getInterfaces():
                    if __interface.getNet() == self:
                        routers.append(__interface.getAddress())
        
        return routers
```

In [ ]:
for asn in base.getAsns():
    __as = base.getAutonomousSystem(asn)
    for net_name in __as.getNetworks():
        __net = __as.getNetwork(net_name)
        print(__net.getDefaultRouter())

10.2.0.254
10.2.1.254
10.2.2.254
10.3.0.254
10.3.1.254
10.3.2.254
10.3.3.254
10.4.0.254
10.4.1.254
10.11.0.254
10.12.0.254
10.150.0.254
10.151.0.254
10.152.0.254
10.153.0.254
10.154.0.254
10.154.1.254
10.160.0.254
10.161.0.254
10.162.0.254
10.163.0.254
10.164.0.254
10.170.0.254
10.171.0.254


同时添加接口 `getDefaultRouterByAsnAndNetwork` 到 `core/emulator.py` 中：

```python
def getDefaultRouterByAsnAndNetwork(self, asn: int, network: str) -> IPv4Address:
        """!
        @brief get the default router for the given AS and network.
        @param asn AS number.
        @param network network name.
        @return IPv4Address of the default router.
        """
        base:Base = self.getLayer('Base')
        return base.getAutonomousSystem(asn).getNetwork(network).getDefaultRouter()
```

In [ ]:
print(emu.getDefaultRouterByAsnAndNetwork(154, "net0"))
print(emu.getDefaultRouterByAsnAndNetwork(154, "net1"))

10.154.0.254
10.154.1.254


4. 是否有 DHCP 服务

经过搜索，发现有 DHCP 服务的网络是含有名为 "DHCP Server" 名字的节点的 Host，最笨的方法就是名字检索是否有这个字段

In [ ]:
for asn in base.getAsns():
    __as = base.getAutonomousSystem(asn)
    for net_name in __as.getNetworks():
        __net = __as.getNetwork(net_name)
        for __node in __net.getAssociations():
            if "dhcp" in __node.getName() or "DHCP" in __node.getName():
                print(f"Net {__net.getPrefix()} has DHCP server")
                break
        

Net 10.151.0.0/24 has DHCP server
Net 10.161.0.0/24 has DHCP server


因此在 `core/Network.py` 中添加接口 `hasDHCPService`（临时）：

```python
def hasDHCPService(self) -> bool:
        """!
        @brief Check if this network has DHCP service.

        @returns true if has DHCP service, false otherwise.
        """
        for __node in self.getAssociations():
            if "dhcp" in __node.getName() or "DHCP" in __node.getName():
                return True
        return False
```

然后在 `core/Emulator.py` 中添加接口 `getDefaultRouterByAsnAndNetwork`（临时）：

```python
def hasDHCPServiceByAsnAndNetwork(self, asn: int, network: str) -> bool:
        """!
        @brief check if the given AS has a DHCP service for the given network.
        @param asn AS number.
        @param network network name.
        @return True if the AS has a DHCP service for the given network.
        """
        base:Base = self.getLayer('Base')
        return base.getAutonomousSystem(asn).getNetwork(network).hasDHCPService()
```

In [ ]:
for asn in base.getAsns():
    __as = base.getAutonomousSystem(asn)
    for net_name in __as.getNetworks():
        __net = __as.getNetwork(net_name)
        if __net.hasDHCPService():
            print(f"Net {__net.getPrefix()} has DHCP server")

print(emu.hasDHCPServiceByAsnAndNetwork(151, "net0"))
print(emu.hasDHCPServiceByAsnAndNetwork(161, "net0"))

开发时发现：

1. `makeStubAsWithHosts` 的 net 名字写死了
2. 有蛮多 `notImplementedError`（是虚函数？不太清楚机理）
3. `pip uninstall seedemu` 没有脚本

Todo:

- [ ] 通过在建立 Emulator 时添加元信息到对象当中的方式来简化 API 的实现（改动较大）